#**$\gamma$ External Radiation Dose**

This article will explain about a method to calculate the radiation dose that exposed through a source outside human body. You can think of it like the sunlight esposure or even your room light exposure. However, this article focused on the gamma radiation since it can be lethal in unsafe dosage. Therefore, the safety of gamma radiation in nuclear or medic application is deemed important.


Hereby, I'll show the script to 

##**1. Packages**

**First**, we need to import several packages such as numpy and pandas. I already summarized the necessary table into an Excel spreadsheet. I'll put the reference of the data in the end of this article. 

You can download the table from this [this URL](https://):

In [ ]:
import math
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile

import requests
url = 'https://'
r   = requests.get(url, allow_redirects=True)
open('', 'wb').write(r.content)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DPR-ProRad.xlsx to DPR-ProRad.xlsx


##**2. Function**

Before we begin the calculation, we need to define the function to calculate our problem. In this second step, I will define several important fuction

###**Linear Interpolation**
The interpolation will help us to find the value that unwritten in the table. In simple way, we can utilize linear interpolation to calculate value between two value written in the table.

$y = f(x)$

$y = \dfrac{(x-x_1)(y_2-y_1)}{(x_2-x_1)} + y_1$


In [ ]:
def Interpolation(x1,y1,x2,y2,x):
  y=(x-x1)*(y2-y1)/(x2-x1)+y1
  return y

###**Gamma Property**

There are some important parameter of radionuclide source that determine the radiation dose. Some radionuclide may emmit multiple energy with varied emmision probability. This function will extract multiple gamma energy of a determined radionuclide in the **spreadsheet's sheet 'Gamma _emmision'**.

where the properties includes:


1.   $f_i$ = energy probability
2.   $E_i$ = emitted energy



In [ ]:
Gammadf = pd.read_excel('DPR-ProRad.xlsx', sheet_name='Gamma_emission',header=[0])

def Gamma_property(Nuclide,Gammadf):
  E=[]; f=[]
  stdev_E=[]; stdev_f=[]
  for i in range(0,len(Gammadf)):
    if Gammadf['Nuclide'].iloc[i]==Nuclide:
	    E.append(Gammadf['E'].iloc[i])
	    stdev_E.append(Gammadf['stdev-E'].iloc[i])
	    f.append(Gammadf['f'].iloc[i])
	    stdev_f.append(Gammadf['stdev-f'].iloc[i])
     #Energy in keV
  return (np.array(E),np.array(f),np.array(stdev_E),np.array(stdev_f))

###**Attenuation & Mass Absorption Coefficient**

Attenuation coefficient is a coefficient that determine the reduction of energy when radiation pass through a medium. On the other hand, Mass Absorption Coefficient determines the radiation mass/energy absorbed by a material when being exposed. Therefore, attenuation coefficeint and Mass Absorption coefficient is based on energy level. The data obtained from **spreadsheet's sheet 'Appendix_D2' for Mass Absorption and 'Appendix_D1' for Attenuation**.

Available Material is:
```
-H Be C N O Na Al Si Fe Cu Pb NaI Ge Air Water Bone Muscle Concrete Pyrex Lucite-
```
If ones used for calculating HVL $\mu_{mass}$ should be converted to $\mu_{lin} \ [cm^{-1}]$ :

###$\mu_{mass}(E_e)=\frac{\mu_{lin}}{\rho[\frac{g}{cm^3}]}$


With Bragg-Law the total from i component attenuation Coefficient is:

###$\mu_{mass}(E_e) [cm^2/g]=\Sigma w_i \mu_{i,e,mass} f_e$


where:


*   $E_e$ = Radiation energy
*   $F_e$ = Emmision probability
*   $_i$  = Medium material-i
*   $w_i$ = Weight fraction of material-i in Medium





In [ ]:
Appendix_D1 = pd.read_excel('DPR-ProRad.xlsx', sheet_name='Appendix D1',header=[0])
Appendix_D2 = pd.read_excel('DPR-ProRad.xlsx', sheet_name='Appendix D2',header=[0])

def Attenuation_coeff(Energy,Fraction,Medium,Ratio,Appendix_D):
  '''
  Mass Attenuation coefficient (Appendix_D1) cm2/g or 
  Mass Absorption coefficient  (Appendix_D2) cm2/g with Units in cgs
  '''
  mu=np.zeros((len(Energy),len(Medium)))
  for m in range(0,len(Medium)):
    for n in range(0,len(Energy)):
      i=0
      while Energy[n]-Appendix_D['E'].iloc[i]>0:
        i+=1
      mu[n,m]=(Interpolation(Appendix_D['E'].iloc[i-1],Appendix_D[Medium[m]].iloc[i-1],
                             Appendix_D['E'].iloc[i],Appendix_D[Medium[m]].iloc[i],Energy[n]))
      print('mu = '+'{:.3e}'.format(mu[n,m])+
            '\tcoeff E = '+'{:.3e}'.format(Energy[n])+' // '+'{:.3e}'.format(Fraction[n]*100)+'%'+
            '\tMedium = '+str(round(np.array(Ratio)[m]*100,2))+' % '+Medium[m])
  M_grid,E_grid=np.meshgrid(np.array(Ratio),Fraction)
  total_attenuation=np.sum(M_grid*E_grid*np.array(mu))
  
  #print('\ntotal_attenuation = '+'{:.3e}'.format(total_attenuation))
  return (np.array(mu)*M_grid,total_attenuation)

###**Buildup factor**

Build up factor is factor that determine the multiplication of radiation dose because of refocused ray after scattered by a shielding material. The value determined by the attenuation factor, thus it also related to radiation energy.

Build Up Factor obtained from **spreadsheet's sheet 'Appendix E'** with normalization equation of:
###**$B=1+a \mu r e^{b \mu r}$**

where :


*   $a$ and $b$ = energy and cosine related  factor
*   $\mu$       = attenuation factor
*   $r$         = shielding thickness


Program will search $a$ and $b$ by checking the table energy value.

```
Available shielding material = Concrete Air Water Iron Lead
```


In [ ]:
Appendix_E = pd.read_excel('DPR-ProRad.xlsx', sheet_name='Appendix E',header=[0])

def buildup(Material,E,mu,r,Appendix_E):
  "Material of shielding = Concrete, Air, Water, Iron, Lead"
  i=0
  while E-Appendix_E['E'].iloc[i]>0:
    i+=1
  a=Interpolation(Appendix_E['E'].iloc[i-1],Appendix_E[Material+'-a'].iloc[i-1],
                  Appendix_E['E'].iloc[i],Appendix_E[Material+'-a'].iloc[i],E)
  b=Interpolation(Appendix_E['E'].iloc[i-1],Appendix_E[Material+'-b'].iloc[i-1],
                  Appendix_E['E'].iloc[i],Appendix_E[Material+'-b'].iloc[i],E)
  B=1+a*mu*r*math.exp(b*mu*r)
  print('a = '+'{:.5e}'.format(a))
  print('b = '+'{:.5e}'.format(b))
  print('B = '+'{:.5e}'.format(B))
  return B

##**3. Calculation**

Now, lets move to the calculation. The basic equation for radiation absorbed dose is written below as function of energy :

###$\dot{D}\ [\frac{J/kg}{MBq/h}](E)=\frac{f(\frac{photons}{d})E(\frac{MeV}{photon})1.6e-13(\frac{J}{MeV})1e6(\frac{d/s}{MBq})3.6e3(\frac{s}{h})\frac{\mu}{m}}{4\pi r^2[m^2]\rho\frac{kg}{m^3}}\times B(E)$

Therefore, if the nuclide emits multiple radiation energy, then the calculation for each energy must be separated and summed at the end.

$^{***}$ This equation define the amount of energy transfered through a surface of the sphere $(4\pi r ^2)$ with radius of the interested point to the source. This equation assume that the source is a isotropic point thus, it used whole surface of spehere.

$\sum_{e=1}^{n}{\dot{D}} = \dot{D}_1+\dot{D}_2+...+\dot{D}_n$



###**Absorbed Dose Calculation ($\dot{D}$)**

The calculation breaks into 3 part which are:

1.   Get the Mass Absorption and Radiation Property
2.   Calculate

In this example we wanted to calculate the absorbed radiation dose emitted from Mo$^{99}$ that passed through perforated concrete (assummed to be 70% concrete and 30% air) and shielded by Lead. The target stand 10cm from the Lead.

Here is the complete illustration of our problem:

*<------ 30 cm perf. concrete ------>[//<------5 mm Lead------->\\]<-----5 cm air ----->{Target}


In [ ]:
Source   = input('Radiation Source is [format = Cs137] = ')
Medium_1 = input('\nRadiation Medium1 is = ')
Medium_r1 = input('Input the Medium1 Composition (w.o)= ')
d_M1     = float(input('Medium1 travel distance (m) = '))
Shield   = input('\nShielding Medium is = ')
Shield_r = input('Input the Shield Composition (w.o)= ')
Shield_R = float(input('Input the Shield Total Density (g/cm3) = '))
d_Sd     = float(input('Shield thickness (m) = '))
Medium_2 = input('\nRadiation Medium2 is = ')
Medium_r2 = input('Input the Medium2 Composition (w.o)= ')
Medium_R2 = float(input('Input the Medium2 Total Density (g/cm3) = '))
d_M2     = float(input('Medium2 travel distance (m) = '))



# ==== input to array ==== 
Medium_1 = Medium_1.split(' ')
Medium_r1 = np.array([float(x) for x in Medium_r1.split(' ')])
Medium_2 = Medium_2.split(' ')
Medium_r2 = np.array([float(x) for x in Medium_r2.split(' ')])
Shield = Shield.split(' ')
Shield_r = np.array([float(x) for x in Shield_r.split(' ')])

Radiation Source is [format = Cs137] = Mo99

Radiation Medium1 is = Concrete Air
Input the Medium1 Composition (w.o)= 0.7 0.3
Medium1 travel distance (m) = 0.3

Shielding Medium is = Lead
Input the Shield Composition (w.o)= 1
Input the Shield Total Density (g/cm3) = 11.34
Shield thickness (m) = 0.005

Radiation Medium2 is = Air
Input the Medium2 Composition (w.o)= 1
Input the Medium2 Total Density (g/cm3) = 1.293e-3
Medium2 travel distance (m) = 0.05


#### **-- target to lead shield**
We need to separate the calculation for source until lead and lead until target. To solve the first part, the absorbed dose equation will be used.

Since the table already in units of $cm^2/g$, therefore, the equation can be simplified into this equation :

$\dot{D}(E)=\frac{f\times E\times 1.6e-13\times 1e6\times 3.6e3\times \mu_{mass}(\frac{cm^2}{s})\times 0.1}{4\pi (1m^2)}\times B(E)$

In [ ]:
# Mass Absorption Property
E,f,stdev_E,stdev_f = Gamma_property(Source,Gammadf)
mu_array=Attenuation_coeff(E/1000,f,Medium_1,Medium_r1,Appendix_D2)[0]

# Sum the coefficient
mu   = np.sum(mu_array,axis=1)

# Finish the equation without Buildup
D_1  = f*E*1.6e-13*1e6*3.6e3*mu*0.1/(np.pi*4)
D_1  = D_1/d_M1**2


print('\n Absorbed Dose in Lead Shielding face is = \n')
for i in range(0,len(D_1)):
  print('\t E'+str(i),' = ',format(D_1[i],'.5E'),' (J/kg)/(MBq h)')

mu = 3.820e-01	coeff E = 4.058e-02 // 1.022e+00%	Medium = 70.0 % Concrete
mu = 3.154e-02	coeff E = 1.405e-01 // 8.960e+01%	Medium = 70.0 % Concrete
mu = 2.900e-02	coeff E = 1.811e-01 // 6.010e+00%	Medium = 70.0 % Concrete
mu = 2.970e-02	coeff E = 3.664e-01 // 1.194e+00%	Medium = 70.0 % Concrete
mu = 2.914e-02	coeff E = 7.395e-01 // 1.212e+01%	Medium = 70.0 % Concrete
mu = 2.899e-02	coeff E = 7.779e-01 // 4.280e+00%	Medium = 70.0 % Concrete
mu = 6.527e-02	coeff E = 4.058e-02 // 1.022e+00%	Medium = 30.0 % Air
mu = 2.470e-02	coeff E = 1.405e-01 // 8.960e+01%	Medium = 30.0 % Air
mu = 2.612e-02	coeff E = 1.811e-01 // 6.010e+00%	Medium = 30.0 % Air
mu = 2.923e-02	coeff E = 3.664e-01 // 1.194e+00%	Medium = 30.0 % Air
mu = 2.908e-02	coeff E = 7.395e-01 // 1.212e+01%	Medium = 30.0 % Air
mu = 2.897e-02	coeff E = 7.779e-01 // 4.280e+00%	Medium = 30.0 % Air

 Absorbed Dose in Lead Shielding face is = 

	 E0  =  6.06174E-06  (J/kg)/(MBq h)
	 E1  =  1.89082E-04  (J/kg)/(MBq h)
	 E2  =  1.55958E-05  


####**-- Buildup by Lead Shield**

Then, we can calculate the dose by weighting the dose in the shielding face with Buildup factor. The value separated based on the energy.



In [ ]:
# Search Attenuation coefficient
mu_shield = Attenuation_coeff(E/1000,f,Shield,Shield_r,Appendix_D1)[0]
mu_shield = np.sum(mu_shield,axis=1) 

# Convert to linear attenuation coefficient
mu_shield = mu_shield*Shield_R

# Calculate buildup and create array of it
build_up=np.zeros((len(E),len(Shield)))
for j in range(0,len(Shield)):
  for i in range(0,len(E)):
    print('\nBuildUp Equation Coefficient for energy','{:.3e}'.format(E[i]),'keV')
    build_up[i,j]=(buildup(Shield[j],E[i]/1000,mu_shield[i],d_Sd,Appendix_E))

# Calculate dose regarding buildup
D_2 = np.sum(D_1*build_up.T,axis=0)
print('\n Absorbed Dose in Lead Shielding end is = \n')
for i in range(0,len(D_2)):
  print('\t E'+str(i),' = ',format(D_2[i],'.5E'),' (J/kg)/(MBq h)')

mu = 1.276e+01	coeff E = 4.058e-02 // 1.022e+00%	Medium = 100.0 % Lead
mu = 2.524e+00	coeff E = 1.405e-01 // 8.960e+01%	Medium = 100.0 % Lead
mu = 1.302e+00	coeff E = 1.811e-01 // 6.010e+00%	Medium = 100.0 % Lead
mu = 2.747e-01	coeff E = 3.664e-01 // 1.194e+00%	Medium = 100.0 % Lead
mu = 9.601e-02	coeff E = 7.395e-01 // 1.212e+01%	Medium = 100.0 % Lead
mu = 8.940e-02	coeff E = 7.779e-01 // 4.280e+00%	Medium = 100.0 % Lead

BuildUp Equation Coefficient for energy 4.058e+01 keV
a = 1.00000e-02
b = -6.48335e-02
B = 1.00690e+00

BuildUp Equation Coefficient for energy 1.405e+02 keV
a = 2.08102e-01
b = 3.01381e-02
B = 1.02991e+00

BuildUp Equation Coefficient for energy 1.811e+02 keV
a = 1.29156e-01
b = -6.19405e-02
B = 1.00949e+00

BuildUp Equation Coefficient for energy 3.664e+02 keV
a = 9.99263e-02
b = -3.53505e-02
B = 1.00156e+00

BuildUp Equation Coefficient for energy 7.395e+02 keV
a = 2.31850e-01
b = -2.05125e-02
B = 1.00126e+00

BuildUp Equation Coefficient for energy 7.779e+02 keV


####**--Lead shield to target**

For the last radiation track, we only need to calculate it using exponential equation. This method was used due to inverse square law. The equation written below:

$\dot{D}(r)=\dot{D}(0) e^{-\mu_{lin} r}$

In [ ]:
# Search attenuation coefficient
m2_array=Attenuation_coeff(E/1000,f,Medium_2,Medium_r2,Appendix_D1)[0]

# Calculate
m2_array=np.sum(m2_array,axis=1)
D_3=D_2*np.exp(-m2_array*d_M2)

print('\n Absorbed Dose in Lead Shielding end is = \n')
for i in range(0,len(D_3)):
  print('\t E'+str(i),' = ',format(D_3[i],'.5E'),' (J/kg)/(MBq h)')

print('\n Total radiation received by targed is = \n\t',format(np.sum(D_3),'.5E'),
      ' (J/kg)/(MBq h)')

mu = 2.271e-01	coeff E = 4.058e-02 // 1.022e+00%	Medium = 100.0 % Air
mu = 1.372e-01	coeff E = 1.405e-01 // 8.960e+01%	Medium = 100.0 % Air
mu = 1.272e-01	coeff E = 1.811e-01 // 6.010e+00%	Medium = 100.0 % Air
mu = 9.889e-02	coeff E = 3.664e-01 // 1.194e+00%	Medium = 100.0 % Air
mu = 7.366e-02	coeff E = 7.395e-01 // 1.212e+01%	Medium = 100.0 % Air
mu = 7.178e-02	coeff E = 7.779e-01 // 4.280e+00%	Medium = 100.0 % Air

 Absorbed Dose in Lead Shielding end is = 

	 E0  =  6.03467E-06  (J/kg)/(MBq h)
	 E1  =  1.93405E-04  (J/kg)/(MBq h)
	 E2  =  1.56441E-05  (J/kg)/(MBq h)
	 E3  =  6.56403E-06  (J/kg)/(MBq h)
	 E4  =  1.32620E-04  (J/kg)/(MBq h)
	 E5  =  4.90286E-05  (J/kg)/(MBq h)

 Total radiation received by targed is = 
	 4.03296E-04  (J/kg)/(MBq h)


###**Equivalent Dose**

For practical purpose, we tend to use equivalent dose that derived from absorbed dose and weighting factor. The weighting factor is a constant derived empirically. This constant determine the damage resulted from ionization radiation as a function of radiation type, energy and radiation target consideration.

###**Simplification by assuming the Radiation move through air**

For practical purpose, usually we used air medium assumption. By doing so the equation become much simpler and easy to used in practice. But the equation is only applicable for air medium. If somehow the medium change or separated by shielding, then this equation can not be used.

###**If ones used Air-Kerma assumption to solve the equation, so this constant will be used:**

$\rho\ \ \ \ \ \ = 1.293\ kg/m^3$

$\Sigma\mu_i\ \ =3.5e-3\ m^{-1} \ |\  60keV<E<2MeV$

$1\ R\ \ = 2.58e-4\ C/kg$

$\frac{J/kg}{C/Kg} \approx 34\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ |\ dry\ air\ at\ 20^{\circ}C;\ 101.325kPa$

if all the constant can be compiled to a variable called Specific Gamma Constant
*The Specific Gamma Constant will Be*

$\Gamma=1.24e-7\Sigma f_i E_i \frac{Sv.m^2}{MBq.h}=3.65e-9\Sigma f_i E_i \frac{(C/kg).m^2}{MBq.h}\approx 0.5\Sigma f_i E_i \frac{R.m^2}{Ci.h}$

$^*$ it is not exactly 0.5 but 0.523 if $2.58e-4\frac{C/kg}{R}$ factor was used in calculation.

$^{**}$ depending on the table data used, mass absorption coefficient may vary.


**Lets compare both method see how similar it is for Co60 source.**

In [ ]:
E,f,stdev_E,stdev_f = Gamma_property('Co60',Gammadf)

####**--Air assumption method**

$\dot{D}=0.5\Sigma f_i E_i (\frac{R.m^2}{Ci.h}) \times 2.58e-4 (\frac{C/kg}{R}) \times \frac{1}{37000} (\frac{Ci}{MBq})\times 34 (\frac{J/kg}{C/kg})$

In [ ]:
D_t1=np.sum(E*f*0.5*2.58e-4/37000*34)
print('Absorbed dose of Co60 is = \n\t',format(D_t1,'.5E'),' (J/kg)/(MBq h)')

Absorbed dose of Co60 is = 
	 3.13532E-05  (J/kg)/(MBq h)


####**--Default calculation**

In [ ]:
mut_array= Attenuation_coeff(E/1000,f,['Air'],[1],Appendix_D2)[0]
mut      = np.sum(mut_array,axis=1)

# Finish the equation without Buildup
D_t2  = f*E*1.6e-13*1e6*3.6e3*mut*0.1/(np.pi*4)

print()
for i in range(0,len(D_t2)):
  print('\t E'+str(i),' = ',format(D_t2[i],'.5E'),' (J/kg)/(MBq h)')

D_t2=np.sum(D_t2)
print('\nAbsorbed dose of Co60 is = \n\t',format(D_t2,'.5E'),
      ' (J/kg)/(MBq h)')

print('\nDifference with assumption = \n\t',format(np.abs(D_t1-D_t2)/D_t2*100,'.3f'),'%')

mu = 6.527e-02	coeff E = 4.058e-02 // 1.022e+00%	Medium = 100 % Air
mu = 2.470e-02	coeff E = 1.405e-01 // 8.960e+01%	Medium = 100 % Air
mu = 2.612e-02	coeff E = 1.811e-01 // 6.010e+00%	Medium = 100 % Air
mu = 2.923e-02	coeff E = 3.664e-01 // 1.194e+00%	Medium = 100 % Air
mu = 2.908e-02	coeff E = 7.395e-01 // 1.212e+01%	Medium = 100 % Air
mu = 2.897e-02	coeff E = 7.779e-01 // 4.280e+00%	Medium = 100 % Air

	 E0  =  1.24090E-07  (J/kg)/(MBq h)
	 E1  =  1.42516E-05  (J/kg)/(MBq h)
	 E2  =  1.30303E-06  (J/kg)/(MBq h)
	 E3  =  5.86201E-07  (J/kg)/(MBq h)
	 E4  =  1.19473E-05  (J/kg)/(MBq h)
	 E5  =  4.42063E-06  (J/kg)/(MBq h)

Absorbed dose of Co60 is = 
	 3.26329E-05  (J/kg)/(MBq h)

Difference with assumption = 
	 3.921 %
